In [2]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Masking
from keras import optimizers
from keras.utils import Sequence
from keras.callbacks import BaseLogger
from keras.models import load_model
from sklearn.metrics import mean_squared_error

import pickle
import pandas as pd
import numpy as np
import math
import itertools
import matplotlib.pyplot as plt
import datetime
from scipy import spatial


Using TensorFlow backend.


In [3]:
VECSIZE = 100

In [4]:
class ShowLoss(tf.keras.callbacks.Callback):
 
    def __init__(self):
        super(ShowLoss, self).__init__()
        self.loss = []

    def on_train_end(self, batch, logs=None):
        fig = plt.figure()
        plt.plot(np.arange(0,len(self.loss)), self.loss, '+', linestyle="-")
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.axis(ymin=0)
        plt.show()
        fig.savefig(f'./loss_{datetime.datetime.now().strftime("%d%m%y_%H:%M:%S")}.png')
    def on_epoch_end(self, epoch, logs=None):
        self.loss.append(logs['loss'])


In [5]:
vectors = pickle.load(open("vectors_doc2vec_sessions.pickle", 'rb'))
#np.random.shuffle(vectors)
_sep = int(len(vectors)*0.7)
_train, _test = vectors[:_sep], vectors[_sep:]
print(len(_train), len(_test))

740 318


In [6]:
def gen_sequences(dataset, vector_size, height = 5):
    while True:
        for _batch in dataset:
            for _h in range(1, len(_batch), height):
                _size = min(height, len(_batch)-_h)
                _x = np.zeros((_size, len(_batch)-1, vector_size))
                _y = np.empty((_size, vector_size))
                for _i in range(0, _size):
                    for _j in range(0, _i+_h):
                        _x[_i][_j] = _batch[_j]
                    _y[_i] = _batch[_i+_h]
                yield _x, _y

for i, seq in enumerate(itertools.islice(gen_sequences([[1,2,3,4], [1,2,3,4,5,6]], 1, height=1), 0, 5)):
    print(i)
    for j, x in enumerate(seq[0]):
        print(x, "  : ", seq[1][j])

0
[[1.]
 [0.]
 [0.]]   :  [2.]
1
[[1.]
 [2.]
 [0.]]   :  [3.]
2
[[1.]
 [2.]
 [3.]]   :  [4.]
3
[[1.]
 [0.]
 [0.]
 [0.]
 [0.]]   :  [2.]
4
[[1.]
 [2.]
 [0.]
 [0.]
 [0.]]   :  [3.]


In [7]:
_args = {"epochs":150, "steps_per_epoch":2000}

nn = Sequential()
nn.add(Masking(mask_value=0., input_shape=(None,VECSIZE)))
nn.add(Bidirectional(LSTM(50, return_sequences=True)))
nn.add(Bidirectional(LSTM(50, return_sequences=True)))
nn.add(Bidirectional(LSTM(50, return_sequences=True)))
nn.add(Bidirectional(LSTM(50, return_sequences=True)))
nn.add(Bidirectional(LSTM(50)))
nn.add(Dense(VECSIZE))
nn.compile(loss='mean_squared_error', optimizer="adam")
nn.summary()

nn.fit_generator(generator=gen_sequences(_train, VECSIZE, height=100), **_args, verbose=1, callbacks=[ShowLoss()])

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, None, 100)         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 100)         60400     
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 100)         60400     
_________________________________________________________________
bidirectional_3 (Bidirection (None, None, 100)         60400     
_________________________________________________________________
bidirectional_4 (Bidirection (None, None, 100)         60400     
_________________________________________________________________
bidirectional_5 (Bidirection (None, 100)               60400     
_________________________________________________________________
dens

KeyboardInterrupt: 

In [ ]:
nn.fit_generator(generator=gen_sequences(_train, VECSIZE, height=100), epochs=50, steps_per_epoch=5000 , verbose=1, callbacks=[ShowLoss()])

In [61]:
nn.evaluate_generator(generator=gen_sequences(_test, VECSIZE, height=100), steps=200)

0.8814283199152911

In [6]:
nn.save("./lstm5x50_sess_bi")

NameError: name 'nn' is not defined

In [7]:
nn = load_model("./lstm5x50_sess_bi")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [8]:
def dist(sampleA, sampleB):
    _e = 0
    for _f in range(len(sampleA)):
        _e =+ pow(sampleA[_f]-sampleB[_f], 2)
    return _e

In [9]:
def sampleDistances(sample):
    #rank : {dist : (batch, sample)}
    _dists = {}
    for _b in range(len(vectors)):
        for _s in range(1, len(vectors[_b])):
            _dist = spatial.distance.cosine(sample, vectors[_b][_s])
            _dists[_dist] = (_b, _s)
    _rank, _r = {}, 0
    for _k in sorted(_dists):
        _r +=1
        _rank[_r] = (_k ,_dists[_k])
    return _rank

def searchRank(_rank, _b, _s):
    for _r in _rank:
        if _rank[_r][1][0] == _b and _rank[_r][1][1] == _s:
            return (_r, _rank[_r][0])
    return None

In [10]:
c, s = 15, 3
rank = sampleDistances(nn.predict(np.array([vectors[c][:s]]))[0])
searchRank(rank, c, s)

(1, 0.08061647415161133)

In [11]:
rank

{1: (0.08061647415161133, (15, 3)),
 2: (0.10147076845169067, (13, 3)),
 3: (0.10200071334838867, (15, 4)),
 4: (0.10421299934387207, (16, 2)),
 5: (0.110390305519104, (12, 4)),
 6: (0.11995655298233032, (16, 1)),
 7: (0.12466716766357422, (14, 1)),
 8: (0.1345452070236206, (11, 2)),
 9: (0.13575607538223267, (15, 5)),
 10: (0.13899236917495728, (18, 3)),
 11: (0.13901585340499878, (17, 2)),
 12: (0.1407794952392578, (13, 1)),
 13: (0.1411571502685547, (14, 2)),
 14: (0.14181441068649292, (8, 3)),
 15: (0.1438809037208557, (18, 1)),
 16: (0.14574742317199707, (12, 3)),
 17: (0.15194886922836304, (14, 5)),
 18: (0.15220677852630615, (18, 2)),
 19: (0.15394467115402222, (17, 1)),
 20: (0.1570783257484436, (12, 5)),
 21: (0.1634678840637207, (10, 3)),
 22: (0.166595458984375, (20, 3)),
 23: (0.1671386957168579, (21, 2)),
 24: (0.1702127456665039, (20, 1)),
 25: (0.17093127965927124, (21, 1)),
 26: (0.17429256439208984, (10, 2)),
 27: (0.17506974935531616, (7, 1)),
 28: (0.1751865148544311

In [30]:
results = []
for _course in range(_sep, len(vectors)):
    for _chapter in range(1, len(vectors[_course])):
        _rank = sampleDistances(nn.predict(np.array([vectors[_course][0:_chapter]]))[0])
        _r = searchRank(_rank, _course, _chapter)
        results.append(_r)
        print(_course, _chapter, " :", _r)

740 1  : (812, 0.33874303102493286)
740 2  : (61, 0.20408934354782104)
740 3  : (103, 0.22500675916671753)
740 4  : (423, 0.2873697876930237)
740 5  : (30, 0.17368388175964355)
740 6  : (449, 0.32399022579193115)
741 1  : (1777, 0.4011446237564087)
741 2  : (910, 0.39030879735946655)
741 3  : (1255, 0.41261428594589233)
741 4  : (699, 0.29150480031967163)
741 5  : (1395, 0.3475794196128845)
741 6  : (789, 0.3012486696243286)
742 1  : (2277, 0.4609242081642151)
742 2  : (1535, 0.47243213653564453)
742 3  : (876, 0.3609697222709656)
742 4  : (502, 0.3575369119644165)
742 5  : (2141, 0.5625)
742 6  : (567, 0.35633713006973267)
743 1  : (1907, 0.49472177028656006)
743 2  : (1904, 0.38845622539520264)
743 3  : (282, 0.305649995803833)
743 4  : (464, 0.36040210723876953)
743 5  : (1601, 0.4149107336997986)
744 1  : (1219, 0.4304301142692566)
744 2  : (1031, 0.36207807064056396)
744 3  : (1608, 0.41645216941833496)
744 4  : (1249, 0.34633511304855347)
745 1  : (719, 0.4213517904281616)
745 2 

798 3  : (3229, 0.48169946670532227)
798 4  : (124, 0.26242315769195557)
799 1  : (2202, 0.6354733109474182)
799 2  : (2092, 0.5541423261165619)
799 3  : (2380, 0.4837648868560791)
799 4  : (1297, 0.3967505693435669)
800 1  : (1475, 0.37299078702926636)
800 2  : (2061, 0.4537721872329712)
800 3  : (1633, 0.37549638748168945)
800 4  : (1331, 0.32656329870224)
801 1  : (227, 0.40843820571899414)
801 2  : (1390, 0.5301898717880249)
801 3  : (2023, 0.49731171131134033)
801 4  : (1209, 0.46721166372299194)
801 5  : (2628, 0.638626366853714)
802 1  : (1454, 0.43424975872039795)
802 2  : (1730, 0.49637091159820557)
802 3  : (2014, 0.4832947850227356)
802 4  : (1710, 0.41131389141082764)
802 5  : (224, 0.36020535230636597)
803 1  : (3078, 0.45785850286483765)
803 2  : (3437, 0.44768625497817993)
803 3  : (3709, 0.6099353432655334)
803 4  : (158, 0.25718891620635986)
804 1  : (1603, 0.5193727016448975)
804 2  : (1787, 0.5538522005081177)
804 3  : (1335, 0.4435310363769531)
804 4  : (1046, 0.395

861 1  : (56, 0.28322112560272217)
861 2  : (171, 0.3184536099433899)
861 3  : (92, 0.30788731575012207)
863 1  : (2990, 0.5607282519340515)
863 2  : (1257, 0.35948240756988525)
863 3  : (2369, 0.4746691584587097)
864 1  : (1634, 0.3743109107017517)
864 2  : (1220, 0.3917679786682129)
864 3  : (540, 0.265811026096344)
864 4  : (464, 0.26232022047042847)
865 1  : (1499, 0.49075591564178467)
865 2  : (2104, 0.4600943922996521)
866 1  : (300, 0.3921249508857727)
866 2  : (570, 0.44474053382873535)
866 3  : (217, 0.335466206073761)
866 4  : (497, 0.3411790132522583)
866 5  : (341, 0.29053056240081787)
866 6  : (174, 0.2941899299621582)
866 7  : (305, 0.23789352178573608)
867 1  : (3982, 0.5170989036560059)
867 2  : (1122, 0.378878116607666)
867 3  : (86, 0.2158041000366211)
868 1  : (612, 0.3781489133834839)
868 2  : (732, 0.36256176233291626)
869 1  : (329, 0.3046169877052307)
869 2  : (1804, 0.39412951469421387)
869 3  : (3299, 0.5270650386810303)
869 4  : (577, 0.27662789821624756)
870 

925 1  : (408, 0.3762027621269226)
925 2  : (702, 0.44218629598617554)
925 3  : (627, 0.35480940341949463)
925 4  : (199, 0.32895737886428833)
925 5  : (745, 0.3328334093093872)
926 1  : (2430, 0.46953922510147095)
926 2  : (1619, 0.44320911169052124)
926 3  : (2153, 0.5546776950359344)
926 4  : (746, 0.49006330966949463)
926 5  : (2389, 0.6082753241062164)
926 6  : (449, 0.3565787076950073)
928 1  : (2441, 0.5089199841022491)
928 2  : (2311, 0.535782128572464)
928 3  : (939, 0.39356446266174316)
928 4  : (2577, 0.5413600206375122)
929 1  : (381, 0.3658151626586914)
929 2  : (1634, 0.4696758985519409)
929 3  : (2991, 0.5558672845363617)
929 4  : (1485, 0.4568069577217102)
929 5  : (1517, 0.38927632570266724)
930 1  : (2155, 0.4219335913658142)
930 2  : (2248, 0.492298424243927)
930 3  : (1116, 0.40156227350234985)
930 4  : (79, 0.22036534547805786)
930 5  : (1230, 0.32197606563568115)
930 6  : (3862, 0.4794765114784241)
931 1  : (2709, 0.5012246668338776)
931 2  : (1938, 0.427156627178

977 2  : (910, 0.34839504957199097)
977 3  : (2201, 0.38426822423934937)
977 4  : (2012, 0.3821806311607361)
978 1  : (941, 0.36982065439224243)
978 2  : (1564, 0.3815845847129822)
978 3  : (1189, 0.43267011642456055)
978 4  : (1798, 0.49335241317749023)
979 1  : (933, 0.3219626545906067)
979 2  : (114, 0.1913241147994995)
979 3  : (1543, 0.37938225269317627)
979 4  : (1419, 0.35511457920074463)
979 5  : (1103, 0.36594921350479126)
980 1  : (3776, 0.5729324817657471)
980 2  : (3048, 0.47026658058166504)
980 3  : (1577, 0.3802186846733093)
981 1  : (3455, 0.4873877763748169)
981 2  : (3534, 0.473486065864563)
981 3  : (3753, 0.5061808526515961)
982 1  : (3625, 0.6013493835926056)
982 2  : (2170, 0.5076326429843903)
982 3  : (2376, 0.4793495535850525)
982 4  : (2472, 0.44951438903808594)
983 1  : (720, 0.3567379117012024)
983 2  : (228, 0.25859057903289795)
983 3  : (1796, 0.38581544160842896)
983 4  : (1803, 0.3944132924079895)
984 1  : (564, 0.33392012119293213)
984 2  : (1122, 0.39282

1030 2  : (1476, 0.5024125277996063)
1030 3  : (829, 0.4526599645614624)
1031 1  : (3976, 0.6687625348567963)
1031 2  : (1130, 0.3540568947792053)
1031 3  : (488, 0.3555369973182678)
1032 1  : (3063, 0.5912796854972839)
1032 2  : (828, 0.4055784344673157)
1032 3  : (358, 0.34652280807495117)
1033 1  : (337, 0.3060288429260254)
1033 2  : (1661, 0.47894996404647827)
1033 3  : (3127, 0.46583813428878784)
1033 4  : (511, 0.3162820339202881)
1033 5  : (474, 0.24212056398391724)
1034 1  : (2423, 0.4988800883293152)
1034 2  : (1256, 0.4341115951538086)
1034 3  : (2755, 0.49171239137649536)
1034 4  : (1562, 0.44603103399276733)
1035 1  : (587, 0.29231637716293335)
1035 2  : (205, 0.23023760318756104)
1035 3  : (1504, 0.34173470735549927)
1035 4  : (1421, 0.3717646598815918)
1036 1  : (757, 0.3928418755531311)
1036 2  : (779, 0.46270668506622314)
1036 3  : (283, 0.3350595235824585)
1036 4  : (550, 0.3725356459617615)
1036 5  : (2188, 0.5078518688678741)
1036 6  : (1048, 0.3580096960067749)
1036

In [31]:
len(results)

1255

In [32]:
results
_sum, _count, _top1, _top10 = 0, 0, 0, 0
for _i in results:
    if _i!=None:
        _sum += _i[0]
        _count+=1
        if _i[0]<=10:
            _top10 += 1
        if _i[0]==1:
            _top1 += 1
_sum/_count, _top1, _top10, _count

(1462.3131474103586, 1, 2, 1255)

In [18]:
resultstrain = []
for _course in range(0, _sep):
    for _chapter in range(1, len(vectors[_course])):
        _rank = sampleDistances(nn.predict(np.array([vectors[_course][0:_chapter]]))[0])
        _r = searchRank(_rank, _course, _chapter)
        resultstrain.append(_r)
        print(_course, _chapter, " :", _r)

0 1  : (56, 0.11667603254318237)
0 2  : (1, 0.09700208902359009)
0 3  : (10, 0.10592740774154663)
0 4  : (11, 0.12050235271453857)
0 5  : (1, 0.11799043416976929)
1 1  : (4, 0.10047328472137451)
1 2  : (1, 0.11742651462554932)
1 3  : (1, 0.08737331628799438)
1 4  : (1, 0.08867210149765015)
2 1  : (6, 0.10543978214263916)
2 2  : (6, 0.13416922092437744)
2 3  : (1, 0.10870623588562012)
2 4  : (1, 0.09509623050689697)
2 5  : (1, 0.07357096672058105)
3 1  : (2, 0.09748756885528564)
3 2  : (1, 0.1081882119178772)
3 3  : (1, 0.09211796522140503)
3 4  : (1, 0.0802653431892395)
4 1  : (1, 0.10367178916931152)
4 2  : (1, 0.04753279685974121)
5 1  : (1, 0.08582675457000732)
5 2  : (1, 0.06157791614532471)
6 1  : (4, 0.10916578769683838)
6 2  : (1, 0.06319534778594971)
6 3  : (1, 0.057874977588653564)
7 1  : (3, 0.09577757120132446)
7 2  : (1, 0.08928543329238892)
7 3  : (1, 0.09339487552642822)
7 4  : (1, 0.09087949991226196)
7 5  : (1, 0.09214669466018677)
8 1  : (1, 0.10154849290847778)
8 2  :

44 1  : (8, 0.0910831093788147)
44 2  : (8, 0.08920246362686157)
44 3  : (4, 0.10569030046463013)
44 4  : (8, 0.13931214809417725)
44 5  : (1, 0.08820527791976929)
44 6  : (1, 0.09320938587188721)
44 7  : (16, 0.10570114850997925)
44 8  : (12, 0.09828662872314453)
45 1  : (2, 0.1055377721786499)
45 2  : (1, 0.10094541311264038)
45 3  : (9, 0.10930901765823364)
45 4  : (37, 0.14153975248336792)
45 5  : (4, 0.08922708034515381)
45 6  : (14, 0.1025192141532898)
45 7  : (55, 0.1379048228263855)
45 8  : (17, 0.10769039392471313)
46 1  : (22, 0.13417047262191772)
46 2  : (5, 0.09994518756866455)
46 3  : (1, 0.10450160503387451)
46 4  : (8, 0.083271324634552)
46 5  : (1, 0.06891292333602905)
46 6  : (7, 0.09038698673248291)
46 7  : (80, 0.15061479806900024)
47 1  : (35, 0.12821346521377563)
47 2  : (24, 0.1251746416091919)
47 3  : (17, 0.13447195291519165)
47 4  : (10, 0.12325215339660645)
47 5  : (6, 0.1319667100906372)
47 6  : (44, 0.12260925769805908)
47 7  : (11, 0.09287989139556885)
47 8

107 1  : (2, 0.10823923349380493)
107 2  : (1, 0.07751047611236572)
107 3  : (1, 0.08894258737564087)
107 4  : (1, 0.07799452543258667)
108 1  : (4, 0.10356074571609497)
108 2  : (1, 0.0792926549911499)
108 3  : (1, 0.0860857367515564)
109 1  : (4, 0.1291191577911377)
109 2  : (1, 0.08286595344543457)
109 3  : (1, 0.07520020008087158)
110 1  : (1, 0.06294089555740356)
110 2  : (1, 0.06692975759506226)
110 3  : (1, 0.09907561540603638)
111 1  : (4, 0.09535729885101318)
111 2  : (1, 0.054028213024139404)
111 3  : (1, 0.05702298879623413)
112 1  : (1, 0.08727484941482544)
112 2  : (1, 0.038510680198669434)
112 3  : (1, 0.06410366296768188)
113 1  : (2, 0.0883711576461792)
113 2  : (1, 0.08275574445724487)
113 3  : (1, 0.07924449443817139)
113 4  : (1, 0.07875734567642212)
114 1  : (1, 0.07038766145706177)
114 2  : (1, 0.07187062501907349)
114 3  : (1, 0.0692288875579834)
115 1  : (1, 0.06078362464904785)
115 2  : (17, 0.12688130140304565)
115 3  : (1, 0.06873118877410889)
115 4  : (1, 0.0

180 4  : (3, 0.07293510437011719)
180 5  : (1, 0.06751608848571777)
180 6  : (1, 0.042432427406311035)
181 1  : (7, 0.06476038694381714)
181 2  : (7, 0.07881778478622437)
181 3  : (8, 0.0705147385597229)
181 4  : (3, 0.06566357612609863)
182 1  : (1, 0.06230366230010986)
182 2  : (27, 0.0867919921875)
182 3  : (1, 0.04489368200302124)
182 4  : (1, 0.052643001079559326)
182 5  : (1, 0.056034743785858154)
182 6  : (10, 0.0861085057258606)
183 1  : (27, 0.1001209020614624)
183 2  : (2, 0.07013189792633057)
183 3  : (2, 0.10684263706207275)
183 4  : (2, 0.07566744089126587)
183 5  : (2, 0.05247849225997925)
184 1  : (28, 0.1122693419456482)
184 2  : (1, 0.08503609895706177)
184 3  : (1, 0.09188365936279297)
185 1  : (1, 0.10194754600524902)
185 2  : (1, 0.08130204677581787)
186 1  : (11, 0.17113900184631348)
186 2  : (1, 0.1072462797164917)
186 3  : (1, 0.12396490573883057)
186 4  : (13, 0.1370803713798523)
186 5  : (2, 0.13310480117797852)
186 6  : (9, 0.14724767208099365)
187 1  : (1, 0.

231 5  : (3, 0.07626765966415405)
232 1  : (2, 0.10804426670074463)
232 2  : (1, 0.10949498414993286)
232 3  : (3, 0.12516945600509644)
232 4  : (1, 0.12651634216308594)
232 5  : (4, 0.14201396703720093)
233 1  : (4, 0.12420147657394409)
233 2  : (1, 0.07489478588104248)
233 3  : (1, 0.06719255447387695)
233 4  : (1, 0.08440542221069336)
233 5  : (6, 0.09474009275436401)
234 1  : (2, 0.10664796829223633)
234 2  : (1, 0.08349591493606567)
234 3  : (1, 0.0764206051826477)
234 4  : (4, 0.1346830129623413)
235 1  : (2, 0.1147581934928894)
235 2  : (1, 0.1253069043159485)
235 3  : (2, 0.13410413265228271)
235 4  : (1, 0.11764192581176758)
235 5  : (4, 0.06879293918609619)
236 1  : (1, 0.08983087539672852)
236 2  : (8, 0.1326373815536499)
236 3  : (1, 0.09831255674362183)
236 4  : (1, 0.13050562143325806)
236 5  : (22, 0.08746331930160522)
237 1  : (24, 0.14239639043807983)
237 2  : (1, 0.10343611240386963)
237 3  : (1, 0.08229851722717285)
237 4  : (1, 0.11394643783569336)
237 5  : (2, 0.06

...

510 6  : (6, 0.09835994243621826)
512 1  : (2, 0.12093514204025269)
512 2  : (1, 0.08482342958450317)
512 3  : (5, 0.05902552604675293)
512 4  : (6, 0.08818960189819336)
513 1  : (147, 0.1713651418685913)
513 2  : (1, 0.11571645736694336)
513 3  : (6, 0.1520688533782959)
513 4  : (12, 0.10707485675811768)
513 5  : (102, 0.14765793085098267)
513 6  : (38, 0.08580011129379272)
513 7  : (5, 0.0704684853553772)
514 1  : (4, 0.07623851299285889)
514 2  : (21, 0.11165052652359009)
514 3  : (9, 0.09139007329940796)
514 4  : (7, 0.11292523145675659)
514 5  : (6, 0.06631803512573242)
514 6  : (10, 0.12241464853286743)
515 1  : (24, 0.11008810997009277)
515 2  : (36, 0.14766281843185425)
515 3  : (2, 0.09302043914794922)
515 4  : (4, 0.10508233308792114)
515 5  : (2, 0.10747307538986206)
515 6  : (9, 0.0851660966873169)
515 7  : (20, 0.08324843645095825)
515 8  : (1, 0.0477367639541626)
515 9  : (3, 0.0940394401550293)
515 10  : (5, 0.08559226989746094)
515 11  : (2, 0.08809375762939453)
515 12 

556 1  : (7, 0.11862784624099731)
556 2  : (7, 0.10832184553146362)
556 3  : (1, 0.0679282546043396)
556 4  : (6, 0.10506165027618408)
556 5  : (4, 0.07336890697479248)
557 1  : (1, 0.0877721905708313)
557 2  : (1, 0.06626546382904053)
557 3  : (1, 0.07966369390487671)
557 4  : (1, 0.07030552625656128)
557 5  : (4, 0.08555424213409424)
558 1  : (1, 0.08668887615203857)
558 2  : (1, 0.0706738829612732)
558 3  : (1, 0.08125805854797363)
558 4  : (7, 0.10512393712997437)
558 5  : (27, 0.0887901782989502)
559 1  : (1, 0.08204734325408936)
559 2  : (4, 0.09875202178955078)
559 3  : (15, 0.13411015272140503)
559 4  : (1, 0.0777512788772583)
559 5  : (5, 0.1041765809059143)
560 1  : (1, 0.07144343852996826)
560 2  : (1, 0.0792878270149231)
560 3  : (2, 0.09775900840759277)
560 4  : (1, 0.08036959171295166)
561 1  : (1, 0.10182249546051025)
561 2  : (1, 0.0877646803855896)
561 3  : (2, 0.10864365100860596)
561 4  : (2, 0.08244937658309937)
562 1  : (1, 0.05590105056762695)
562 2  : (2, 0.09270

614 1  : (3, 0.09794920682907104)
614 2  : (2, 0.09877777099609375)
614 3  : (1, 0.09286117553710938)
614 4  : (3, 0.11152905225753784)
615 1  : (11, 0.13692837953567505)
615 2  : (2, 0.10510605573654175)
615 3  : (1, 0.09306728839874268)
616 1  : (1, 0.09789377450942993)
616 2  : (1, 0.08350944519042969)
616 3  : (1, 0.07710075378417969)
617 1  : (1, 0.08513933420181274)
617 2  : (1, 0.07186543941497803)
617 3  : (1, 0.058277904987335205)
617 4  : (1, 0.07026928663253784)
618 1  : (1, 0.09575885534286499)
618 2  : (1, 0.06451588869094849)
618 3  : (1, 0.07619619369506836)
618 4  : (1, 0.08120036125183105)
619 1  : (1, 0.09088718891143799)
619 2  : (1, 0.07939779758453369)
620 1  : (1, 0.10011464357376099)
620 2  : (1, 0.0711851716041565)
620 3  : (1, 0.0813552737236023)
621 1  : (1, 0.08254849910736084)
621 2  : (2, 0.08915406465530396)
621 3  : (1, 0.06191205978393555)
622 1  : (24, 0.14878374338150024)
622 2  : (2, 0.09613198041915894)
622 3  : (1, 0.05748569965362549)
622 4  : (2, 

677 1  : (1, 0.10735148191452026)
677 2  : (1, 0.06060713529586792)
677 3  : (1, 0.06681978702545166)
678 1  : (1, 0.08854132890701294)
678 2  : (1, 0.0905921459197998)
678 3  : (1, 0.09476453065872192)
679 1  : (1, 0.07802784442901611)
679 2  : (1, 0.09553360939025879)
679 3  : (1, 0.09396117925643921)
679 4  : (1, 0.1017046570777893)
680 1  : (1, 0.09583616256713867)
680 2  : (3, 0.11382180452346802)
680 3  : (1, 0.07239067554473877)
680 4  : (1, 0.0793808102607727)
681 1  : (6, 0.12504714727401733)
681 2  : (2, 0.10948199033737183)
681 3  : (1, 0.10897725820541382)
681 4  : (3, 0.1292901635169983)
681 5  : (1, 0.09914606809616089)
681 6  : (2, 0.11851835250854492)
682 1  : (6, 0.111350417137146)
682 2  : (1, 0.08518147468566895)
683 1  : (9, 0.10946780443191528)
683 2  : (4, 0.09442269802093506)
683 3  : (1, 0.06303161382675171)
683 4  : (1, 0.06827956438064575)
684 1  : (1, 0.08238112926483154)
684 2  : (1, 0.07198059558868408)
684 3  : (1, 0.080966055393219)
684 4  : (1, 0.0947507

726 5  : (1, 0.06800514459609985)
726 6  : (3, 0.10153895616531372)
726 7  : (3, 0.09647727012634277)
727 1  : (5, 0.11728304624557495)
727 2  : (1, 0.08119595050811768)
727 3  : (10, 0.11611872911453247)
727 4  : (4, 0.11117613315582275)
727 5  : (1, 0.07498955726623535)
727 6  : (1, 0.08332931995391846)
727 7  : (1, 0.0975918173789978)
728 1  : (5, 0.1153072714805603)
728 2  : (7, 0.10694420337677002)
728 3  : (7, 0.10496789216995239)
728 4  : (1, 0.06390959024429321)
728 5  : (4, 0.08783799409866333)
729 1  : (1, 0.07477045059204102)
729 2  : (1, 0.08704197406768799)
729 3  : (1, 0.0519486665725708)
729 4  : (1, 0.06840711832046509)
730 1  : (2, 0.09715312719345093)
730 2  : (1, 0.0928429365158081)
730 3  : (1, 0.0715172290802002)
730 4  : (1, 0.09309864044189453)
731 1  : (1, 0.09449726343154907)
731 2  : (1, 0.06849056482315063)
731 3  : (2, 0.1042642593383789)
731 4  : (4, 0.10027605295181274)
732 1  : (1, 0.09721493721008301)
732 2  : (6, 0.10952335596084595)
732 3  : (1, 0.0571

In [21]:
resultstrain
_sum, _count, _top1, _top10 = 0, 0, 0, 0
for _i in resultstrain:
    if _i!=None:
        _sum += _i[0]
        _count+=1
        if _i[0]<=10:
            _top10 += 1
        if _i[0]==1:
            _top1 += 1
_sum/_count, _top1, _top10, _count

(6.592719352831363, 1734, 2786, 3214)

In [20]:
_c = 0
for _course in range(0, len(vectors)):
    for _chapter in range(len(vectors[_course])):
        _c += 1
_c

5485